## Install Dependencies, Import Libraries, Clone Github, and Load Data

In [ ]:
# Install Dependenies
!pip install -q -U langchain langchain_community transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
# Import Libraries
import os
from google.colab import userdata
import pandas as pd
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import time

In [ ]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/CVX_Rice_project.git

Cloning into 'CVX_Rice_project'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 143 (delta 40), reused 0 (delta 0), pack-reused 62
Receiving objects: 100% (143/143), 57.48 MiB | 11.97 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
text_df = pd.read_csv('CVX_Rice_project/Text_Data/Consolidated_Text_Data.csv')
display(text_df.head())

,Ticker,Date,Article Headline,Article Text
0,CVX,"May 13, 2024 10:05 PM",Chevron Corporation,"Stock Report | March 12, 2022 | NYSE Symbol: C..."
1,CVX,"May 13, 2024 03:29 PM",CFRA LIFTS VIEW ON SHARES OF CHEVRON CORPORATI...,"Stock Report | August 05, 2023 | NYSE Symbol: ..."
2,CVX,"May 11, 2024 06:00 PM",Chevron Corporation,"Stock Report | March 04, 2023 | NYSE Symbol: C..."
3,CVX,"May 04, 2024 05:49 PM",Chevron Corporation,"Stock Report | October 17, 2020 | NYSE Symbol:..."
4,CVX,"May 01, 2024 10:03 PM",Chevron Corporation,"Stock Report | December 25, 2021 | NYSE Symbol..."


## Load Llama 3

In [ ]:
# Define Quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# Load Model and Tokenizer
model_4bit = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Create Hugging Face Pipeline
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=5000, # Need to find the max tokens per article
        truncation=True,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id)

llm = HuggingFacePipeline(pipeline=pipeline_inst)

In [ ]:
# Prompt and response function
template = """<s> Given the text from a financial news article, analyze the content and produce a structured summary categorizing the company's performance and related impacts across multiple predefined categories. Each category should be listed with a corresponding sentiment derived from the article. If a category is not mentioned or relevant based on the article content, mark it as 'N/A'. Ensure that all categories are addressed for a comprehensive summary.

        Categories and Sentiment Options to be Analyzed:
        •	Financial Targets
            •	Greatly Exceeded Target
            •	Exceeded Target
            •	Achieved Target
            •	Below Target
            •	Significantly below Target
        •	Exploration / Discoveries
            •	Major Discovery
            •	Minor Discovery
            •	Favorable Exploration Results
            •	Unfavorable Exploration Results
            •	Exploration Failure
        •	Reserves
            •	Significant Reserves Add
            •	Minor Reserves Add
            •	Stable Reserve Levels
            •	Small Reserves Loss / Writeoff
            •	Significant Reserves Depletions / Writeoff
        •	Production Targets
            •	Greatly Exceeded Target
            •	Exceeded Target
            •	Achieved Target
            •	Below Target
            •	Significantly below Target
        •	New Energy Investments / Projects
            •	Major Advancements in New Energy Initiatives
            •	Minor Advancements in New Energy Initiatives
            •	Setback in New Energies Project
            •	New Energy Projects Abandoned or Failed
        •	Acquisitions and Mergers
            •	Major Acquisition or Merger
            •	Minor Acquisition or Merger
            •	Delay in Acquisition or Merger
            •	Cancelled Acquisition or Merger
        •	Divestments
            •	Major Divestment
            •	Small Divestment
        •	Public Sentiment
            •	Very Positive
            •	Positive
            •	Neutral
            •	Negative
            •	Very Negative
        •	Regulatory / Geopolitical Factors
            •	Favorable change to Operations
            •	Potential Large Disruption to Operations
            •	Potential Small Disruption to Operations
        •	Environmental Factors
            •	Very Positive
            •	Positive
            •	Neutral
            •	Negative
            •	Very Negative

        Output Format:
        •	Use bullet points for each category.
        •	For each category, provide the status or sentiment extracted from the article. If no specific information is available, indicate 'N/A'.

        Example Output:
            •	Financial Targets - Achieved Target
            •	Exploration / Discoveries - N/A
            •	Reserves - Minor Reserves Add
            •	Production Targets - Greatly Exceeded Target
            •	New Energy Investments / Projects - Major Advancements in New Energy Initiatives
            •	Acquisitions and Mergers - Major Acquisition or Merger
            •	Divestments - N/A
            •	Public Sentiment - Very Positive
            •	Regulatory / Geopolitical Factors - Potential Small Disruption to Operations
            •	Environmental Factors - Neutral

        Task:
        •	Parse the article text to identify relevant data points and sentiments for each category.
        •	Summarize the findings in the specified output format.
        •	Maintain accuracy and clarity in categorization and summary presentation.

        **Constraints:** ONLY RESPOND USING THE PROVIDED FORMAT/n/n

        The text from the financial news article is below:
        {article} </s>
        """

def generate_response(article):
  prompt = PromptTemplate(template=template, input_variables=["article"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"article":article})
  return response

In [ ]:
# Test Function on first article
article = text_df.iloc[0]['Article Text']
full_response = generate_response(article)
split_response = full_response.split("</s>", 1)
final_response = split_response[1]
print(final_response)


         ```
        **Output:**
        •	Financial Targets - Achieved Target
        •	Exploration / Discoveries - N/A
        •	Reserves - Minor Reserves Add
        •	Production Targets - Greatly Exceeded Target
        •	New Energy Investments / Projects - Major Advancements in New Energy Initiatives
        •	Acquisitions and Mergers - N/A
        •	Divestments - N/A
        •	Public Sentiment - Very Positive
        •	Regulatory / Geopolitical Factors - Potential Small Disruption to Operations
        •	Environmental Factors - Neutral
        **Note:** The article does not mention specific information about the company's reserves, exploration, or mergers and acquisitions, so the corresponding categories are marked as 'N/A'. The company's financial targets, production targets, and new energy investments are highlighted as 'Achieved Target', 'Greatly Exceeded Target', and 'Major Advancements in New Energy Initiatives', respectively. The public sentiment is reported as 'Very Posit

In [ ]:
# Add sentiment categories to DF
sentiment_categories = {'Financial Targets' : None,
                        'Exploration / Discoveries' : None,
                        'Reserves' : None,
                        'Production Targets' : None,
                        'New Energy Investments / Projects' : None,
                        'Acquisitions and Mergers' : None,
                        'Divestments' : None,
                        'Public Sentiment' : None,
                        'Regulatory / Geopolitical Factors' : None,
                        'Environmental Factors' : None}
text_df = text_df.assign(**sentiment_categories)
display(text_df.head())

,Ticker,Date,Article Headline,Article Text,Financial Targets,Exploration / Discoveries,Reserves,Production Targets,New Energy Investments / Projects,Acquisitions and Mergers,Divestments,Public Sentiment,Regulatory / Geopolitical Factors,Environmental Factors
0,CVX,"May 13, 2024 10:05 PM",Chevron Corporation,"Stock Report | March 12, 2022 | NYSE Symbol: C...",None,None,None,None,None,None,None,None,None,None
1,CVX,"May 13, 2024 03:29 PM",CFRA LIFTS VIEW ON SHARES OF CHEVRON CORPORATI...,"Stock Report | August 05, 2023 | NYSE Symbol: ...",None,None,None,None,None,None,None,None,None,None
2,CVX,"May 11, 2024 06:00 PM",Chevron Corporation,"Stock Report | March 04, 2023 | NYSE Symbol: C...",None,None,None,None,None,None,None,None,None,None
3,CVX,"May 04, 2024 05:49 PM",Chevron Corporation,"Stock Report | October 17, 2020 | NYSE Symbol:...",None,None,None,None,None,None,None,None,None,None
4,CVX,"May 01, 2024 10:03 PM",Chevron Corporation,"Stock Report | December 25, 2021 | NYSE Symbol...",None,None,None,None,None,None,None,None,None,None


In [ ]:
# Test Adding results to DF
lines = final_response.strip().split('\n')

for category in sentiment_categories:
    for line in lines:
        if category in line:
            _, sentiment = line.split(' - ', 1)
            print(sentiment)
            text_df.loc[0, category] = sentiment
            break

display(text_df.head(1))

Achieved Target
N/A
Minor Reserves Add
Greatly Exceeded Target
Major Advancements in New Energy Initiatives
N/A
N/A
Very Positive
Potential Small Disruption to Operations
Neutral


,Ticker,Date,Article Headline,Article Text,Financial Targets,Exploration / Discoveries,Reserves,Production Targets,New Energy Investments / Projects,Acquisitions and Mergers,Divestments,Public Sentiment,Regulatory / Geopolitical Factors,Environmental Factors
0,CVX,"May 13, 2024 10:05 PM",Chevron Corporation,"Stock Report | March 12, 2022 | NYSE Symbol: C...",Achieved Target,N/A,Minor Reserves Add,Greatly Exceeded Target,Major Advancements in New Energy Initiatives,N/A,N/A,Very Positive,Potential Small Disruption to Operations,Neutral


## Loop through articles and append results to df

In [ ]:
# Test first 5 articles
start_time = time.time()

for index, article in enumerate(text_df['Article Text'].iloc[:5]):
    # Get LLM response
    full_response = generate_response(article)
    split_response = full_response.split("</s>", 1)
    final_response = split_response[1]
    print(index)
    print(final_response)
    print()

    # Split into lines
    lines = final_response.strip().split('\n')

    for category in sentiment_categories:
        for line in lines:
            if category in line:
                _, sentiment = line.split(' - ', 1)
                text_df.loc[index, category] = sentiment
                break

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")
display(text_df.head(5))

0

         """

        # Extract the article text
        article_text = text

        # Initialize the categories and sentiments
        financial_targets = 'N/A'
        exploration_discoveries = 'N/A'
        reserves = 'N/A'
        production_targets = 'N/A'
        new_energy_investments_projects = 'N/A'
        acquisitions_mergers = 'N/A'
        divestments = 'N/A'
        public_sentiment = 'N/A'
        regulatory_geopolitical_factors = 'N/A'
        environmental_factors = 'N/A'

        # Parse the article text
        for line in article_text.splitlines():
            # Check for financial targets
            if 'capital spending' in line:
                financial_targets = 'Below Target'
            elif 'capital spending over the next five years would range from $15B - $17B' in line:
                financial_targets = 'Achieved Target'
            elif 'likely being towards the low end of the range' in line:
                financial_targets = 'Below Target'
       

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
display(text_df.head(5))